In [25]:
# If PEPit is not installed yet, you can run this cell.
!pip install pepit==0.0.2

# Code description
This code aims at verifying (numerically) the inequality from Lemma 4.1 of the paper
> "Last-Iterate Convergence of Optimistic Gradient Method for Monotone
       Variationnal Inequalities".

## Problem setup:
Consider the problem of finding a zero of a monotone Lipschitz operator:
       $$ \text{find } x\in Q \text{ such that } \langle F(x);y-x\rangle \geq 0 \text{ for all }y\in Q$$
where $F$ is monotone and $L$-Lipschitz, and $Q$ is convex and compact.

## Algorithm: 
The past extragradient method is described by two sets of iterates:
$x^k$, $\tilde{x}^k$ where $k$ denotes the iteration counter, as follows:

- Initialize $\tilde{x}^0 = x^0$, and $x^1=\mathrm{Proj}_Q(x^0 - \gamma  F(x^0))$, then run
- for $k=1,...,N-1$:
     $$\tilde{x}^k    = \mathrm{Proj}_Q(x^k - \gamma  F(\tilde{x}^{k-1}))$$
     $$ x^{k+1} = \mathrm{Proj}_Q(x^k - \gamma  F(\tilde{x}^k))$$

## Potential function:
Denotes $p_k := \|x^k - x^{k-1}\|^2+ \| x^k - x^{k-1} - 2 \gamma (F(x^k) - F(\tilde{x}^{k-1})) \|^2$ the code compute the maximum (i.e., worst-case) value of
 $$ p_{k+1} - p_k + (1-5 L^2 \gamma^2) \|x^{k+1}-\tilde{x}^k\|^2 + \gamma^2 \|F(x^{k+1})-F(\tilde{x}^k)\|^2,$$
which should always be "$\cdot\leq 0$" for verifying the identity from Lemma 4.1. In the code below, we use k = 1 for notational convenience.

In [26]:
# Run this code before executing the cell below
from math import sqrt
import numpy as np
from PEPit import PEP
from PEPit.operators import LipschitzStronglyMonotoneOperator
from PEPit.functions import ConvexIndicatorFunction
from PEPit.primitive_steps import proximal_step

In [48]:
##########################################################
# parameters: MODIFY HERE!

# pick the parameters for which you want to verify
# the inequality (numerically)
L = 1;
gamma = 10;

##########################################################

# verbose & verification tolerance options
verbose = 0;
tolerance = 1e-5;

# (0) Initialize an empty PEP
problem = PEP()

# (1) Set up the problem class
L  =  L; mu = 0; # F is 1-Lipschitz and 0-strongly monotone
F    = problem.declare_function(LipschitzStronglyMonotoneOperator, param={'L': L, 'mu': mu})
indQ = problem.declare_function(ConvexIndicatorFunction, param={'D': np.inf})

# (2) Set up the starting points
tx0 = problem.set_initial_point() # this is \tilde{x}^0
x0  = problem.set_initial_point() # this is x^0

# (3) Run the algorithm
x1,_,_       = proximal_step(x0 - gamma * F.gradient(tx0),indQ,1);
tx1,_,_      = proximal_step(x1 - gamma * F.gradient(tx0),indQ,1); 
x2,_,_       = proximal_step(x1 - gamma * F.gradient(tx1),indQ,1);

#  define the expressions (recall that our objective is to verify that
#  p2 - p1 + residual<=0 for all F and sequence generated by the
#  past extragradient method.
p1 = (x1-x0)**2 + (x1-x0-2*gamma*(F.gradient(x1)-F.gradient(tx0)))**2;
p2 = (x2-x1)**2 + (x2-x1-2*gamma*(F.gradient(x2)-F.gradient(tx1)))**2;
residual = (1-5*L**2*gamma**2)*(x2-tx1)**2 + gamma**2 * (F.gradient(x2)-F.gradient(tx1))**2;

# (4) Set up the performance measure 
expression_to_verify = p2 - p1 + residual;
problem.set_performance_metric(expression_to_verify);

# (5) Solve the PEP
worstcase_value = problem.solve(verbose=verbose)


# (6) is the potential verified? Success if expression2 - expression1 <= 0 for the 
#     choice of the parameters above.
print('Did PEPit verify the potential (within prescribed numerical precision)? {:}  \t'.format(worstcase_value<tolerance))
worstcase_value # this should be close to zero

Did PEPit verify the potential (within prescribed numerical precision)? True  	


2.8230715543031693e-09